# DQN on Breakout



In [1]:
%%javascript

Jupyter.keyboard_manager.command_shortcuts.add_shortcut('r', {
    help : 'run all cells',
    help_index : 'zz',
    handler : function (event) {
        IPython.notebook.execute_all_cells();
        return false;
    }
});

<IPython.core.display.Javascript object>

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
%matplotlib inline

In [16]:
import tensorflow as tf
import gym
import keras 

Using TensorFlow backend.


In [4]:
## Hyperparameters - volatile

gamma = 0.99


In [13]:
## Hyperparameters - stable
env_name = 'Breakout-v0'
env_name = 'CartPole-v0'

architecture = []
architecture.append(['c', 32, 8, 4])
architecture.append(['c', 64, 4, 2])
architecture.append(['c', 64, 3, 1])
architecture.append(['d', 512])



In [14]:
## Derived settings

env = gym.make(env_name)
obs = env.reset()
s_size = list(obs)
a_size = env.action_space.n
architecture.append(['d', a_size])

[2017-06-12 14:12:01,598] Making new env: CartPole-v0


In [15]:
## Create model

modelInput = tf.placeholder(dtype=tf.float32, shape=[None] + s_size, name='modelInput')

model = Sequential()
with tf.name_scope('model'):
    for layer in architecture:
        print(layer)
        if layer[0] == 'c':
            
        

['c', 32, 8, 4]
['c', 64, 4, 2]
['c', 64, 3, 1]
['d', 512]
['d', 2]
